In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.linalg import lu
from scipy.linalg import lu_solve
from scipy.linalg import lu_factor

#  Präsenzaufgabenstuff
# A = np.array([[1e-9,0.5],[0.5,0.5]])
# b = np.array([0.5,0.25])
# lu,piv = lu_factor(A)

# x = lu_solve((lu,piv),b)
# print(x)
# np.dot(A,x)

# 2
## 2.1
<p>
Bringe die gegebene Matrix auf obere 3-Ecksgestalt. Wir nehmen an, dass das LGS eindeutig lösbar ist und damit alle $b_i$ nicht null sind. Unser LGS sieht wie folgt aus:
</p>
&nbsp;

<math>\begin{align}
b_1x_1+c_1x_2&=y_1; & i&=1\\
a_ix_{i-1}+b_ix_i+c_ix_{i+1}&=y_i; & i&=2,...,N-1\\
a_Nx_{N-1}+b_Nx_N&=y_N; & i&=N\\
\,\end{align}</math>

<p>
Wir modifizieren die 2. Gleichng wie folgt:
</p>
<p>
$$(Gleichung\,2)\cdot b_1 - (Gleichung\, 1) \cdot a_2$$

</p>
also:

<p>
<math>\begin{align}
(a_2 x_1 + b_2 x_2  + c_2 x_3) b_1 - (b_1 x_1  + c_1 x_2) a_2 & = y_2 b_1 - y_1 a_2 \\
(b_2 b_1 - c_1 a_2) x_2  + c_2 b_1 x_3 & = y_2 b_1 - y_1 a_2
\,\end{align}</math>
​
</p>

wie man daran sieht, fällt der $x_1$ Term raus. Analog erhält man aus der modifizierten 2. und der 3. Gleichung:

<p>
    
<math>\begin{align}
(a_3 x_2 + b_3 x_3 + c_3 x_4) (b_2 b_1 - c_1 a_2) -
((b_2 b_1 - c_1 a_2) x_2 + c_2 b_1 x_3) a_3
& = y_3 (b_2 b_1 - c_1 a_2) - (y_2 b_1 - y_1 a_2) a_3
\\
(b_3 (b_2 b_1 - c_1 a_2) - c_2 b_1 a_3 )x_3 + c_3 (b_2 b_1 - c_1 a_2) x_4
& = y_3 (b_2 b_1 - c_1 a_2) - (y_2 b_1 - y_1 a_2) a_3
\,\end{align}</math>
</p>    

<p>
Wie man wieder sieht, fällt der $x_2$-Term wieder raus. Dies mann man nun Analog bis zum ende fortsetzen. Man erhält:
​
</p>
<p>
<math>\begin{align}
\tilde{a}_i&=0\\
\tilde{b}_1&=b_1\\
\tilde{b}_i&=b_i \cdot \tilde{b}_{i-1}- \tilde{c}_{i-1}a_i\\
\tilde{c}_1&=c_1\\
\tilde{c}_i&=c_i \tilde{b}_{i-1}\\
\tilde{y}_1&=y_1\\
\tilde{y_i}&=y_i\tilde{b}_{i-1}-\tilde{y}_{i-1}a_i
\,\end{align}</math>
</p>

## 2.2
Jetzt haben wir eine obere Dreiecksmatirx mit $\tilde b_i$ auf der Hautpdiagonalen und $\tilde c_i$ auf der Nebendiagonalen. Damit ist
<p>
<math>
    \begin{align}
    x_N&=\frac{y_N}{\tilde{b}_N}\\
    x_i&=\frac{\tilde{y_i}-\tilde{c_i} x_{i+1}}{b_{i-1}}; \qquad i=N-1, N-2, ..., 1
    \,\end{align}
</math>
</p>

## 2.3

Nun implementieren wir unseren Algorithmus für gegebenes a, b, c wobei dies die Koeffizientenvektoren sind und Lösungsvektor y.

In [2]:
# Iterative Gaußelimination
# Backward Substituion
def trielim(a,b,c,y):
    ''' Funktion, die für eine gegebene 3-Ecksmatrix und einen Verktor x den Vektor y ausrechent (Ax=y)
        Inputs:
            a      - (N-1) x 1 numpy array, Einträge obere Nebendiagonale
            b      - N x 1 numpy array, Einträge Hauptdiagonale
            c      - (N-1) x 1 numpy array, Einträge untere Nebendiagonale
            y      - N x 1 numpy array, Ergebnis y
        Output:
            x      - N x 1 numpy array, Lösungsvektor x
            
        Helpvariables (only arrays):
            h      - entspricht b tilde aus 2.1
            i      - entspricht c tilde aus 2.1
            j      - entspricht y tilde aus 2.1
            rev    - Lösungseintrage x in umgekehrter Reihenfolge
    '''
    h=np.zeros(len(b))
    i=np.zeros(len(c))
    j=np.zeros(len(y))
    h[0]=b[0]
    i[0]=c[0]
    j[0]=y[0]
    k=1
    
    # Schleife zum berechnen der tilde koeffizienten
    while k < len(b):
        if(k!=len(b)-1):
            i[k]=c[k]*h[k-1]
        h[k]=b[k]*h[k-1]-i[k-1]*a[k-1]
        j[k]=y[k]*h[k-1]-j[k-1]*a[k-1]
        k=k+1
    
    rev = np.zeros(len(y))
    rev[0] = j[-1]/h[-1]
    m = 1
    
    #Schleife zur berechnung der x_i, jedoch noch Rückwärts
    while m < len(h):
        rev[m]=(j[-(1+m)]-i[-m]*rev[m-1])/h[-(1+m)]
        m=m+1
    
    s = 0
    x=np.zeros(len(y))
    
    # schleife zum umdrehen des Ergebnisvektors, getrennt zum leichteren Verständnis des Codes. 
    while s < len(rev):
        x[s]=rev[-(s+1)]
        s=s+1
    return x


## 2.4
Wir nehmen nun N = 10 und setze alle Einträge von a auf -1, von b auf 3, von c auf -1 und von y auf 0.2

In [3]:
'''
Inputs:
            a      - (N-1) x 1 numpy array, Einträge obere Nebendiagonale
            b      - N x 1 numpy array, Einträge Hauptdiagonale
            c      - (N-1) x 1 numpy array, Einträge untere Nebendiagonale
            y      - N x 1 numpy array, Ergebnisvektor y
'''

N = 10
a = np.ones(N-1)*-1
b = np.ones(N)*3
c = np.ones(N-1)*-1
y = np.ones(N)*0.2

print('Lösungsvektor x =',trielim(a,b,c,y))

Lösungsvektor x = [0.12359551 0.17078652 0.18876404 0.19550562 0.19775281 0.19775281
 0.19550562 0.18876404 0.17078652 0.12359551]


## 2.5

Wir prüfen nun nach, ob der berechnete Lösungsvektor bei Multiplikation mit der Matrix, den geforderten y-Vektor liefert.

In [4]:
def triprod(a,b,c,x):
    '''
Funktion, die für eine gegebene 3-Ecksmatrix und einen Verktor x den Vektor y ausrechent (Ax=y)
        Inputs:
            a      - (N-1) x 1 numpy array, Einträge obere Nebendiagonale
            b      - N x 1 numpy array, Einträge obere Nebendiagonale
            c      - (N-1) x 1 numpy array, Einträge obere Nebendiagonale
            x      - N x 1 numpy array, Vektor x
        Output:
            y      - N x 1 numpy array, Ergebnisvektor y
'''
    y = np.zeros(len(x))
    y[0]=b[0]*x[0]+c[0]*x[1]
    t=1
    while t<len(x)-1:
        y[t]=a[t-1]*x[t-1]+b[t]*x[t]+x[t+1]*c[t]
        t=t+1
    y[-1]=a[-1]*x[-2]+b[-1]*x[-1]
    return y


In [5]:
# Einsetzen des Lösungsvektors aus obiger Funktion in die Matrixmultiplikation mit A

triprod(a,b,c,trielim(a,b,c,y))

array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

Man stellt sofort fest, dass der berechnete Lösungsvektor tatsächlich den gegeben y-Vektor bei Multiplikation mit der Matrix, die aus den drei Diagonalen a,b und c besteht zurückliefert. Das Ergebnis ist sehr zufriedenstellend.